In [1]:
%run ../../Utils/yp_utils.py

# Initial setup

In [2]:
paper_pmid = 29549835
paper_name = 'ng_chia_2018' 

In [3]:
datasets = pd.read_csv('extras/YeastPhenome_' + str(paper_pmid) + '_datasets_list.txt', sep='\t', header=None, names=['pmid', 'name'])

In [4]:
datasets.set_index('pmid', inplace=True)

# Load & process the data

In [5]:
original_data = pd.read_excel('raw_data/Supplementary Worksheet (EJMC 2018).xlsx', sheet_name='HOP_peptide 50 ')

In [6]:
print('Original data dimensions: %d x %d' % (original_data.shape))

Original data dimensions: 4817 x 8


In [7]:
original_data['ORF'] = original_data['ORF'].astype(str)

In [8]:
# Eliminate all white spaces & capitalize
original_data['ORF'] = clean_orf(original_data['ORF'])

In [9]:
# Translate to ORFs 
original_data['ORF'] = translate_sc(original_data['ORF'], to='orf')

In [10]:
# Make sure everything translated ok
t = looks_like_orf(original_data['ORF'])
print(original_data.loc[~t,])

                   ORF GENE  logFC  logCPM   LR  PValue  FDR  \
index_input                                                    
4589         YBR160WAS  NaN    NaN     NaN  0.0     1.0  1.0   

             Gene  Description  
index_input                     
4589                       NaN  


In [11]:
original_data = original_data.loc[t,:]

In [12]:
original_data.set_index('ORF', inplace=True)
original_data.index.name='orf'

In [13]:
original_data = original_data.groupby(original_data.index).mean()

In [14]:
original_data.shape

(4791, 5)

In [15]:
original_data.head()

,logFC,logCPM,LR,PValue,FDR
orf,,,,,
YAL002W,-1.692550,5.649546,11.180089,0.000827,0.098870
YAL004W,0.079785,8.154021,0.091888,0.761790,1.000000
YAL005C,0.267255,8.783332,1.187574,0.275819,0.933282
YAL007C,-0.438804,7.986629,2.488096,0.114710,0.766158
YAL008W,0.469239,7.731517,3.288013,0.069787,0.660744


# Prepare the final dataset

In [16]:
data = original_data[['logFC']].copy()

In [17]:
dataset_ids = [16443]
datasets = datasets.reindex(index=dataset_ids)

In [18]:
lst = [datasets.index.values, ['value']*datasets.shape[0]]
tuples = list(zip(*lst))
idx = pd.MultiIndex.from_tuples(tuples, names=['dataset_id','data_type'])
data.columns = idx

In [19]:
data.head()

dataset_id,16443
data_type,value
orf,
YAL002W,-1.692550
YAL004W,0.079785
YAL005C,0.267255
YAL007C,-0.438804
YAL008W,0.469239


## Subset to the genes currently in SGD

In [20]:
genes = pd.read_csv(path_to_genes, sep='\t', index_col='id')
genes = genes.reset_index().set_index('systematic_name')
gene_ids = genes.reindex(index=data.index.values)['id'].values
num_missing = np.sum(np.isnan(gene_ids))
print('ORFs missing from SGD: %d' % num_missing)

ORFs missing from SGD: 22


In [21]:
data['gene_id'] = gene_ids
data = data.loc[data['gene_id'].notnull()]
data['gene_id'] = data['gene_id'].astype(int)
data = data.reset_index().set_index(['gene_id','orf'])

In [22]:
data.head()

,dataset_id,16443
,data_type,value
gene_id,orf,
2,YAL002W,-1.692550
1863,YAL004W,0.079785
4,YAL005C,0.267255
5,YAL007C,-0.438804
6,YAL008W,0.469239


In [23]:
data.shape

(4769, 1)

# Normalize

In [24]:
data_norm = normalize_phenotypic_scores(data, has_tested=True)

In [25]:
# Assign proper column names
lst = [datasets.index.values, ['valuez']*datasets.shape[0]]
tuples = list(zip(*lst))
idx = pd.MultiIndex.from_tuples(tuples, names=['dataset_id','data_type'])
data_norm.columns = idx

In [26]:
data_norm[data.isnull()] = np.nan
data_all = data.join(data_norm)

In [27]:
data_all.head()

dataset_id          16443          
data_type           value    valuez
gene_id orf                        
2       YAL002W -1.692550 -1.260404
1863    YAL004W  0.079785  0.072515
4       YAL005C  0.267255  0.213506
5       YAL007C -0.438804 -0.317500
6       YAL008W  0.469239  0.365412

# Print out

In [28]:
for f in ['value','valuez']:
    df = data_all.xs(f, level='data_type', axis=1).copy()
    df.columns = datasets['name'].values
    df = df.droplevel('gene_id', axis=0)
    df.to_csv(paper_name + '_' + f + '.txt', sep='\t')

# Save to DB

In [29]:
from IO.save_data_to_db3 import *

In [30]:
save_data_to_db(data_all, paper_pmid)

  0%|          | 0/1 [00:00<?, ?it/s]

Deleting all datasets for PMID 29549835...
Inserting the new data...


100%|██████████| 1/1 [00:07<00:00,  7.22s/it]

Updating the data_modified_on field...
